In [8]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
dataset = pd.read_csv('train.csv')
# Drop irrelevant data labels into the training
dataset = dataset.drop(['PassengerId', 'Name'], axis=1)
# Convert number to float type
dataset['CryoSleep'] = dataset['CryoSleep'].astype(float)
dataset['VIP'] = dataset['VIP'].astype(float)
dataset['Transported'] = dataset['Transported'].astype(float)
label_encoders = {}
# Encoding categorical columns ('HomePlanet', 'Cabin', 'Destination') to numeric values
for column in ['HomePlanet', 'Cabin', 'Destination']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column]).astype(float)
    label_encoders[column] = le
dataset.fillna(0, inplace=True)
dataset.info()
# Splitting the dataset into features and target variable
X = dataset.drop("Transported", axis=1).values
y = dataset["Transported"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   float64
 1   CryoSleep     8693 non-null   float64
 2   Cabin         8693 non-null   float64
 3   Destination   8693 non-null   float64
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   float64
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   float64
dtypes: float64(12)
memory usage: 815.1 KB


In [3]:
# Defining a grid of hyperparameters for the random forest classifier
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt']
}
# Initializing GridSearchCV with the random forest classifier and the hyperparameters grid
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           verbose=2,
                           n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Hyperparameters: {best_params}")
print(f"Best Score (Mean Cross-Validation Accuracy): {best_score}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


C:\Users\xalzh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 100}
Best Score (Mean Cross-Validation Accuracy): 0.8027937551355793


In [4]:
# Defining a grid of hyperparameters for the random forest classifier
param_grid = {
    'n_estimators': [80, 90, 100, 110, 120],
    'max_depth': [8, 9, 10, 11, 12],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto', 'sqrt']
}
# Initializing GridSearchCV with the random forest classifier and the hyperparameters grid
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           verbose=2,
                           n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Hyperparameters: {best_params}")
print(f"Best Score (Mean Cross-Validation Accuracy): {best_score}")

Fitting 5 folds for each of 150 candidates, totalling 750 fits


C:\Users\xalzh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best Hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 110}
Best Score (Mean Cross-Validation Accuracy): 0.8037797863599014


In [18]:
# Initializing the RandomForestClassifier with the best hyperparameters obtained from grid search
clf = RandomForestClassifier(n_estimators=110,
                             min_samples_leaf=1,
                             max_depth=10,
                             max_features='auto',
                             random_state=42)

clf.fit(X_train, y_train)
train_preds = clf.predict(X_train)
val_preds = clf.predict(X_val)
train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)

print(f"Training Accuracy: {train_acc}")
print(f"Validation Accuracy: {val_acc}")

C:\Users\xalzh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Training Accuracy: 0.8532456861133936
Validation Accuracy: 0.7925613496932515


In [19]:
# Importing the test dataset from the CSV
dataset = pd.read_csv('test.csv')
dataset = dataset.drop(['PassengerId', 'Name'], axis=1)
dataset['CryoSleep'] = dataset['CryoSleep'].astype(float)
dataset['VIP'] = dataset['VIP'].astype(float)
dataset['Transported'] = dataset['Transported'].astype(float)
label_encoders = {}
for column in ['HomePlanet', 'Cabin', 'Destination']:
    le = LabelEncoder()
    dataset[column] = le.fit_transform(dataset[column]).astype(float)
    label_encoders[column] = le
dataset.fillna(0, inplace=True)
dataset.info()
X = dataset.drop("Transported", axis=1).values
X_train = torch.tensor(X, dtype=torch.float32)
train_preds = clf.predict(X_train)
y_pred_binary = [True if prob == 1. else False for prob in train_preds]
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['Transported'] = y_pred_binary
sample_submission_df.to_csv('sample_submission.csv', index=False)
sample_submission_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   float64
 1   CryoSleep     4277 non-null   float64
 2   Cabin         4277 non-null   float64
 3   Destination   4277 non-null   float64
 4   Age           4277 non-null   float64
 5   VIP           4277 non-null   float64
 6   RoomService   4277 non-null   float64
 7   FoodCourt     4277 non-null   float64
 8   ShoppingMall  4277 non-null   float64
 9   Spa           4277 non-null   float64
 10  VRDeck        4277 non-null   float64
 11  Transported   4277 non-null   float64
dtypes: float64(12)
memory usage: 401.1 KB


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
